In [2]:
# Monitoring NOAA Website for Vulnerabilities

# Objective: Scrape NOAA’s web pages to check for outdated software or potential security vulnerabilities.

import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import ssl
import socket
from datetime import datetime


In [3]:

# Function to check for SSL certificate expiration
def check_ssl_expiration(domain):
    try:
        context = ssl.create_default_context()
        with socket.create_connection((domain, 443)) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as ssock:
                cert = ssock.getpeercert()
                expiration_date = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y %Z")
                return expiration_date
    except Exception as e:
        print(f"Error checking SSL for {domain}: {e}")
        return None

# Function to check for outdated plugins or libraries
def check_outdated_software(soup):
    outdated_plugins = []
    scripts = soup.find_all('script')
    for script in scripts:
        if script.get('src'):
            # Example: Checking for jQuery version (simplified check)
            if 'jquery' in script['src'].lower():
                version = script['src'].split('/')[-1].replace('.js', '')
                if version < '3.5.0':  # Assume 3.5.0 as the threshold for outdated versions
                    outdated_plugins.append(('jQuery', version))
    return outdated_plugins

# Function to find unprotected forms
def check_unprotected_forms(soup):
    unprotected_forms = []
    forms = soup.find_all('form')
    for form in forms:
        if not form.get('action') or 'https' not in form['action']:
            unprotected_forms.append(form)
    return unprotected_forms

# URL of the NOAA page to scrape
url = 'https://www.weather.gov/'

# Send a request to the website
response = requests.get(url)
domain = urlparse(url).netloc

# Check SSL certificate expiration
ssl_expiration = check_ssl_expiration(domain)
if ssl_expiration:
    print(f"SSL Certificate for {domain} expires on {ssl_expiration}")
    if ssl_expiration < datetime.now():
        print("Warning: SSL certificate has expired!")
else:
    print("Could not retrieve SSL information.")

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Check for outdated software
outdated_software = check_outdated_software(soup)
if outdated_software:
    print("Outdated software found:")
    for plugin, version in outdated_software:
        print(f"{plugin} version {version}")
else:
    print("No outdated software detected.")

# Check for unprotected forms
unprotected_forms = check_unprotected_forms(soup)
if unprotected_forms:
    print(f"Found {len(unprotected_forms)} unprotected forms:")
    for form in unprotected_forms:
        print(form)
else:
    print("No unprotected forms detected.")



SSL Certificate for www.weather.gov expires on 2025-06-10 23:59:59
No outdated software detected.
Found 2 unprotected forms:
<form action="/widgets/MyForecastWidget.php" id="myfcst-form">
<span id="myfcst-title">Customize Your <strong>Weather.gov</strong></span>
<div id="myfcst-input-block">
<input id="myfcst-location-input" name="myfcst-location-input" type="text" value="City, ST"/>
<label for="myfcst-location-input">Enter Your City, ST or ZIP Code</label><br/>
<input id="myfcst-remember" name="myfcst-remember" type="checkbox"/>
<label for="myfcst-remember">Remember Me</label>
</div>
<div class="myfcst-footer">
<input id="myfcst-submit" type="submit" value="Get Weather"/>
<p class="myfcst-policy"><a href="/privacy.php">Privacy Policy</a></p>
</div>
</form>
<form name="jump">
<select name="menu" size="1">
<option value="https://alerts-v2.weather.gov">Warnings By State</option>
<option value="https://alerts-v2.weather.gov">Warnings By County</option>
<option> </option>
<option value="ht